In [1]:
import pandas as pd

In [2]:
x = pd.read_csv('advml_proj.csv')

In [3]:
x.rename(columns={'0':'topic_cat','1':'probability'},inplace=True)

In [7]:
x.set_index(x.id+x.probability, inplace=True)
x.head()

,Unnamed: 0,topic_cat,probability,id,complaint_type,derived_topic
3.343537e+06,0,0,0.072703,3343537,Checking or savings account,"credit card, bank america, checking account, a..."
3.343537e+06,1,2,0.039928,3343537,Checking or savings account,"* ~, section *, credit reporting, * usc, colle..."
3.343537e+06,2,3,0.086291,3343537,Checking or savings account,"collection agency, never received, debt ~, col..."
3.343537e+06,3,4,0.134954,3343537,Checking or savings account,"customer service, ~ called, called ~, ~ told, ..."
3.343537e+06,4,5,0.018079,3343537,Checking or savings account,"reporting agency, consumer reporting, identity..."


In [9]:
y = x.pivot(index="id", columns="topic_cat", values="probability").max(axis=1).reset_index().rename(columns={0:'probability'})
y['top_cat'] = 1
y.set_index(y.id+y.probability, inplace=True)
y.head()

,id,probability,top_cat
1.290198e+06,1290198,0.148697,1
1.290200e+06,1290200,0.199466,1
1.290304e+06,1290304,0.164490,1
1.290308e+06,1290308,0.166727,1
1.290316e+06,1290316,0.230529,1


In [13]:
z = y.join(x, how='left',lsuffix='_y')

#pd.merge(y,x,how="left",on=['id','probability'])

In [19]:
z.groupby(['complaint_type','topic_cat'],as_index=True)['top_cat'].agg('sum').reset_index().to_csv('topics_complaints_eval.csv')
